In [30]:
!pip install spotipy
!pip install XlsxWriter
import os
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
os.environ['SPOTIPY_CLIENT_ID']='c98e011c0f6a4d5b8ed0cf4caef922c4'
os.environ['SPOTIPY_CLIENT_SECRET']='058ecc9c70a04bb4acdf19673e96af29'
spotify=spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

In [31]:
df=pd.read_csv('drive/My Drive/MoodPlaylists.csv')

In [32]:
mainsongsdf=pd.DataFrame(columns={'emotion','track','album','artists','popularity','url','imageurl'})

In [33]:
for i in range(len(df)):
    emo=df.loc[i]['emotion']
    id=df.loc[i]['playlistid']
    results=spotify.playlist(id,additional_types=('track',))
    playdict={'emotion':emo,'track':[],'album':[],'artists':[],'popularity':[],'url':[],'imageurl':[]}
    for i in results['tracks']['items']:
        playdict['track'].append(i['track']['name'])
        playdict['album'].append(i['track']['album']['name'])
        s=''
        for j in i['track']['artists']:
            s=s+j['name']+', '
        playdict['artists'].append(s[:len(s)-2])
        playdict['popularity'].append(i['track']['popularity'])
        playdict['url'].append(i['track']['external_urls']['spotify'])
        playdict['imageurl'].append(i['track']['album']['images'][1]['url'])
    playdf=pd.DataFrame(playdict)
    playdf=playdf.sort_values(by=['popularity'],ascending=False).reset_index(drop=True)
    mainsongsdf=mainsongsdf.append(playdf)
mainsongsdf=mainsongsdf.reset_index(drop=True)

In [34]:
print(mainsongsdf)

                         artists  ...                  track
0                            TLC  ...              No Scrubs
1                        Beyoncé  ...          Irreplaceable
2                  Tracy Chapman  ...     Give Me One Reason
3        Blu Cantrell, Sean Paul  ...  Breathe - Rap Version
4                   Janis Joplin  ...               Cry Baby
...                          ...  ...                    ...
1227                    Sheppard  ...               Geronimo
1228              Colbie Caillat  ...  Brighter Than The Sun
1229  Owl City, Carly Rae Jepsen  ...              Good Time
1230            American Authors  ...    Best Day Of My Life
1231             Black Eyed Peas  ...        I Gotta Feeling

[1232 rows x 7 columns]


In [35]:
writer=pd.ExcelWriter('drive/My Drive/EmotionSongs.xlsx',engine='xlsxwriter')
mainsongsdf.to_excel(writer,sheet_name='Sheet1',index=False)
writer.save()